## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ["T9", "T13", "T12", "T2", "T25", "T14", "T29", "T15", "T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
value_counts = application_df["CLASSIFICATION"].value_counts()
filtered_values = value_counts[value_counts < 1883]
unique_values = filtered_values.index
unique_values

Index(['C7000', 'C1700', 'C4000', 'C5000', 'C1270', 'C2700', 'C2800', 'C7100',
       'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300', 'C1240', 'C8000',
       'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200', 'C1238', 'C1278',
       'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100', 'C1257', 'C1600',
       'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246', 'C1267', 'C1256',
       'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248', 'C6100', 'C1820',
       'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245', 'C2500', 'C1570',
       'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120', 'C8210', 'C2561',
       'C4500', 'C2150'],
      dtype='object')

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = unique_values

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(application_df, columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"])
converted_df = application_df.copy()
converted_df = converted_df.drop("IS_SUCCESSFUL", axis=1)
converted_df = pd.concat([converted_df, dummies], axis=1)
converted_df = converted_df.drop(columns=["APPLICATION_TYPE", "AFFILIATION", "CLASSIFICATION", "USE_CASE", "ORGANIZATION", "INCOME_AMT", "SPECIAL_CONSIDERATIONS"], axis=1)
converted_df.head()

,STATUS,ASK_AMT,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,5000,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,108590,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,1,5000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,6692,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,142590,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
features = converted_df.drop("IS_SUCCESSFUL", axis=1)
target = converted_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    # Allow kerastuner to decide which activation function to use in hidden layers
#     activation = hp.Choice('activation', ("relu, sigmoid, softmax, softplus, softsign, tanh, selu, elu, exponential").split(", "))
    activation = hp.Choice('activation',["sigmoid", "softmax", "linear", "relu", "leakyrelu", "elu"])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=len(features.columns)))
    
    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        activation = hp.Choice('activation', ["sigmoid", "softmax", "linear", "relu", "leakyrelu", "elu"])
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [13]:
# Import the kerastuner library
import keras_tuner as kt

# Create a `Hyperband()` tuner instance
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
)

Using TensorFlow backend
Reloading Tuner from .\untitled_project\tuner0.json


In [14]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

In [15]:
# Train each of the top models for 100 epochs
for model in tuner.get_best_models(3):
    model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5664 - accuracy: 0.7302 - val_loss: 0.5662 - val_accuracy: 0.7318
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5654 - accuracy: 0.7310 - val_loss: 0.5650 - val_accuracy: 0.7318
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5640 - accuracy: 0.7308 - val_loss: 0.5641 - val_accuracy: 0.7320
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5623 - accuracy: 0.7308 - val_loss: 0.5629 - val_accuracy: 0.7303
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5613 - accuracy: 0.7301 - val_loss: 0.5619 - val_accuracy: 0.7308
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5601 - accuracy: 0.7312 - val_loss: 0.5606 - val_accuracy: 0.7315
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5593 - accuracy: 0.7308 - val_loss: 0.5605 - val_accuracy: 0.7320

In [45]:
nn = tf.keras.models.Sequential()

# 1st hidden layer
nn.add(tf.keras.layers.Dense(units=100, input_dim=len(features.columns), activation="relu"))

# 2nd hidden layer
nn.add(tf.keras.layers.Dense(units=50, input_dim=len(features.columns), activation="relu"))

# 3rd hidden layer
nn.add(tf.keras.layers.Dense(units=50, input_dim=len(features.columns), activation="relu"))

# 4th hidden layer
nn.add(tf.keras.layers.Dense(units=50, input_dim=len(features.columns), activation="relu"))

# 5th hidden layer
nn.add(tf.keras.layers.Dense(units=25, input_dim=len(features.columns), activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_58 (Dense)            (None, 100)               4600      
                                                                 
 dense_59 (Dense)            (None, 50)                5050      
                                                                 
 dense_60 (Dense)            (None, 50)                2550      
                                                                 
 dense_61 (Dense)            (None, 50)                2550      
                                                                 
 dense_62 (Dense)            (None, 25)                1275      
                                                                 
 dense_63 (Dense)            (None, 1)                 26        
                                                                 
Total params: 16051 (62.70 KB)
Trainable params: 1605

In [46]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5679 - accuracy: 0.7213
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5559 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5521 - accuracy: 0.7315
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5507 - accuracy: 0.7311
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5496 - accuracy: 0.7330
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5484 - accuracy: 0.7341
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5479 - accuracy: 0.7326
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5470 - accuracy: 0.7338
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5462 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5463 - accura

In [48]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_v3.h5")

268/268 - 0s - loss: 0.5597 - accuracy: 0.7293 - 259ms/epoch - 966us/step
Loss: 0.5596673488616943, Accuracy: 0.7293294668197632
